# User Guide

The purpose of this guide is to explain the basic concepts of the *scarlet* package and how they are used. We also show how they can be extended and customized for more specialized science cases.
The [API Documentation](api_docs.rst) contains more detailed descriptions of the modules and classes used in *scarlet*, and a more rigorous overview of the mathematics and algorithms used by *scarlet* is described in [Melchior et al. 2018](https://arxiv.org/abs/1802.10157).

## Basic Concepts and Structure

The goal of *scarlet* is to create a model of individual astrophysical sources from a collection of observations of a blended region of the sky. These observations can be in multiple filter bands (which we will call "channels" internally), telescopes with different resolutions, and eventually even spectroscopic instruments. As is pointed out by Robert Lupton, perfect reconstruction of a blended scene is [impossible](https://docushare.lsst.org/docushare/dsweb/Services/Document-29071), however by making a few minor assumptions *scarlet* improves on other blending algorithms by leveraging as much data as possible.

The basic assumption of *scarlet* is that the sources in an astrophysical image can be thought of as a collection of multiple [Component](component.ipynb#scarlet.component.Component) objects, where each component has a single morphology (shape) that is the same in each band (SED). In this model more complicated objects, like galaxies, can be thought of as a combination multiple components (a [ComponentTree](component.ipynb#scarlet.component.ComponentTree)), where components with different SEDs represent different populations of stars or gases in the host galaxy. In order to properly separate sources further assumptions are required, for example assuming that all of the flux is positive and that stars and galaxies monotonically decrease from their centers. The use of constraints is discussed in more detail in [Updates and Constraints](#Updates-and-Constraints).

A [Frame](observation.ipynb#scarlet.observation.Frame) contains the metadata for the hyperspectral cube *scarlet* seeks to construct by fitting one [Observation](observation.ipynb#scarlet.observation.Observation) or several observations. Each observation can have multiple filter bands with a different PSF in each band that is internally matched to the target PSF of the model. This allows the model to be convolved into each observed frame for comparison with the input data.

Finally the [Blend](blend.ipynb#scarlet.blend.Blend) class contains the `frame`, `observations`, and `sources` (a list of [Component](component.ipynb#scarlet.component.Component) or [ComponentTree](component.ipynb#scarlet.component.ComponentTree) objects) and executes the optimization algorithm. [Blend](blend.ipynb#scarlet.blend.Blend) also implements the minimization algorithm described in [Melchior et al. 2018](https://arxiv.org/abs/1802.10157), which is briefly described below.

The deblending algorithm forms a model of the scene

$$\mathsf{M}= \sum_{k=1}^K \mathsf{A}_k^T \times \mathsf{S}_k = \mathsf{A}\mathsf{S}, $$

where $\mathsf{A}_k \in \mathbb{R}^B$ is the normalized SED and $\mathsf{S}_k \in \mathbb{R}^N$ is the morphology of a single component in the model with $B$ bands and $N$ pixels in each band.
It is important to note that this matrix factorization implies that SEDs and morphologies are independent, e.g. the SED of a component does not change over the region covered by its morphology.

The scene is fit by minimizing the likelihood of the model, namely minimizing

$$f(\mathsf{A},\mathsf{S}) = \frac{1}{2} || \mathsf{Y}-\mathsf{A}\mathsf{S} ||_2^2, $$

where $\mathsf{Y}$ is an image cube and $||.||_2$ is the element-wise $L_2$ (Frobenius) norm.
Each component $k$ can have $M_k$ different constraint functions $g_{km}$, equivalent to minimizing
$$f(\mathsf{A}, \mathsf{S}) + \sum_{k=1}^K \sum_{m=1}^{M_k} g^A_{km} \left(\mathsf{A}_{km} \right) + g^S_{km} \left(\mathsf{S}_{km} \right)$$

Those constraints are applied to each source in the form of proximal operators, a handy mathematical approach for imposing non-smooth constraints that (if properly formulated) are guaranteed to converge; the curious reader will find more details in [Parikh & Boyd 2014](http://www.web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf) and [Combettes & Pesquet 2011](https://link.springer.com/chapter/10.1007/978-1-4419-9569-8_10).
In short, proximal operators map an input vector to the nearest vector that satisfied the respective constraint.
Many constraints/penalty functions have analytic proximal operators. 

Note that the PSF of the model $\mathsf{M}$ above is the target PSF of the [Scene](observation.ipynb#scarlet.observation.Scene), meaning the likelihood we evaluate is more complicated than the one shown above due to an extra PSF convolution performed in each band with the difference kernel to match the seeing in each [Observation](observation.ipynb#scarlet.observation.Observation) filter band. *scarlet* uses the [autograd](https://github.com/HIPS/autograd) package to calculate the gradient of this more complicated model in each iteration during optimization.

## Import packages and load data

We need to load an example image (here an image cube with 5 bands) *and* a detection catalog.
If such a catalog is not available packages like [SEP](http://sep.readthedocs.io/) and [photutils](https://photutils.readthedocs.io/en/stable/) will happily generate one, but for this example we use part of the detection catalog generated by the [LSST DM stack](https://github.com/lsst).
While not fundamentally needed, some *scarlet* sources have improved initialization if the background noise levels are known as well, so we also estimate the background RMS of the images:

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

import numpy as np
import scarlet
from astropy.visualization.lupton_rgb import AsinhMapping, LinearMapping

# Load the sample images
data = np.load("../data/hsc_cosmos_35.npz")
images = data["images"]
channels = data["filters"]
psfs = data["psfs"]
catalog = data["catalog"]

# Estimate of the background noise level, here we're lazy
bg_rms = np.ones((len(psfs),), dtype=images.dtype) * .1

## Frame and Observation

[Frame](observation.ipynb#scarlet.observation.Frame) and [Observation](observation.ipynb#scarlet.observation.Observation) are usually the first two objects you create. 

`Frame` defines the characteristic of a hyperspectral data cube. At its core, it is the `shape` of the cube, for which we use the convention `(C, Ny, Nx)` for the number of elements in 3 dimensions: `C` for the number of bands/channels and `Ny, Nx` for the number of pixels at every channel.

Additionally, you can and often must provide an image cube of the PSF model (one image per channel), an `astropy.WCS` structure to translate from pixel to sky coordinates, and labels for all channels. The reason for specifying them is to enable the code to internally map from the model frame, in which you seek to fit a model, to the observed data frame.

**It crictical to realize that there are two frames:** one for model that reconstructs the scene on the sky, and one for any observation of the scene. They may be identical, but mostly they are not, in which case the observation is an information-reduced rendering of the model. The loss of information could be in terms of signal-to-noise ratio, spatial resolution, PSF blurring, spectral coverage, or all of the above. 

In our example, we do not have PSF-matched images, so we need to define a virtual PSF for your model frame that is narrower or equal to the best seeing PSF in all of the observations. When combining ground and space based data, it is appropriate to use the PSF of the space telescope.

For example, we can define the model PSF using

In [ ]:
# Define the pixel grid
X = np.arange(psfs.shape[2])
Y = np.arange(psfs.shape[1])
X, Y = np.meshgrid(X, Y)
coords = np.stack([Y, X])
y0, x0 = (psfs.shape[1]-1) // 2, (psfs.shape[2]-1) // 2
# Use a narrow gaussian as the target PSF
model_psf = scarlet.psf.gaussian(coords, y0, x0, 1, .9)
# Normalize the target_psf so that the input flux is not rescaled
model_psf /= model_psf.sum()
model_psf = model_psf[None,:,:]

`Observation` is a hyperspectral data cube, with an optional cube for weights. Internally, it'll construct its own `Frame`.
Assuming that our observations are all from a single camera and that observation and the model frame use the same pixel scale, we can initialize them by

In [ ]:
frame = scarlet.Frame(images.shape, psfs=model_psf, channels=channels)
observation = scarlet.Observation(images, psfs=psfs, channels=channels).match(frame)

The previous command calls the `match` method to compute the PSF difference kernel; because the channels are identical, no spectral matching will be performed.

We generally recommend this pattern:
1. define model frame
2. construct observation
3. match it to the model frame

For more details on matching for observations at different resolutions, see [Multi-Resolution Deblending](tutorials/multiresolution.ipynb).

## Component and Source

As described [above](#Basic-Concepts-and-Structure), a [Component](component.ipynb#scarlet.component.Component) is the fundamental unit of a *scarlet* model, characterized by a 2D morphology and a 1D spectrum. A component must therfore be initialized with a `frame`, a `sed` and a `morph` (morphology) arguments.

As a (marginally useful) example we construct a source with a single pixel at the center. It reads off the SED from an observation at that location:

In [ ]:
# The (y, x) location of the source
center = (int(catalog["y"][0]), int(catalog["x"][0]))
# Get the SED for each observation at the location of the central pixel
sed = scarlet.source.get_pixel_sed(center, observation)
# Set the morphology such that only the central pixel has any intensity
morph = np.zeros((frame.Ny, frame.Nx))
morph[center] = 1
comp = scarlet.Component(frame, sed, morph)

Additional arguments for `Component` can fix SED and morpology so that they are not updated during the fit, and provide a [Prior](component.ipynb#scarlet.component.Prior), i.e. a constraint applied as an an update to the gradient of the likelihood for the given component. While not yet incorporated into the master branch of *scarlet*, experimental code already uses real galaxy morphologies as priors. We recommend watching the [scarlet github repo](https://github.com/fred3m/scarlet) for its release in the near future.

For a more practical approach, we introduce the concept of **sources**, which are single or multiple components joined into a logical unit. For instance, resolved galaxies may have a bulge component and a disk component, or even more advanced components like a jet or dust lanes. 

For many applications, the built-in sources

* [PointSource](source.ipynb#scarlet.source.PointSource)
* [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource)
* [MultiComponentSource](source.ipynb#scarlet.source.MultiComponentSource)

will be perfectly suitable. But since components encode the characteristics of the sources we seek to separate, they can be fully customized with initialization and [update](user_docs.ipynb#Updates-and-Constraints) methods.

In the case of a known star it might also be useful to fix the morphology and allow only the amplitude in each band to be updated by fixing the morphology:

In [ ]:
point_source = scarlet.PointSource(frame, center, observation, fix_morph=True)

See [Point Source Tutorial](tutorials/point_source.ipynb) for information on using point sources to fit crowded stellar fields with *scarlet*. But most of the time the type of source is unknown and could potentially be an extended object, like a galaxy. In that case it is useful to initialize each source as an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) that is initialized to be both symmetric and monotonic about its center.

### Creating Sources

One caveat is that *scarlet* does not do any detection and requires an input catalog for each source. Because most object detection algorithms operate in a single band, possibly with some merging of catalogs between bands and epochs, there is a tradeoff between tweaking a detection algorithm to detect the faintest sources in a blend vs trying to minimize spurrious detections of noise or wings of bright sources. Undetected objects can have adverse effects on their detected neighbors, meaning it is often better to crank up detection at the risk of having more spurrious objects.

A consequence of this behavior is that there are likely to be some detected peaks that don't have significant flux above the noise level, resulting in an inability of *scarlet* to initialize them and causing a [SourceInitError](source.ipynb#scarlet.source.SourceInitError). How you handle these barely deteted objects will be specific to your science case, however one option to attempt to separate truly spurrious detections from barely detected sources is:

In [ ]:
sources = []
for src in catalog:
    try:
        # Try to initialize the source as an extended object based on the estimated background RMS
        new_source = scarlet.ExtendedSource(frame, (src['y'],src['x']), observation, bg_rms)
    except scarlet.SourceInitError:
        # The source may not have had sufficient flux above the predicted background
        # to be initialized as an extended source, but there might be enough flux
        # at the center in one or more bands to initialize it as a point source
        try:
            new_source = scarlet.PointSource(frame, (src['y'], src['x']), observation)
        except scarlet.SourceInitError:
            # None of the default scarlet classes can handle this particular source
            # Either you will need to create your own inherited class for initialization or
            # ignore this source
            print("Could not initialze source at {0}".format((src['y'], src['x'])))
            continue
    sources.append(new_source)

### Displaying a model

Before we start to constrain and fit the model this is a useful time to learn how to display a scarlet model. [Lupton et al. 2004](https://iopscience.iop.org/article/10.1086/382245) showed the advantage of using the `arcsinh` function to display astrophysical images with a proper normalization to preserve colors while showing objects with a wide range of fluxes. A slightly modified version of their algorithm

$$f(x) = \frac{1}{Q} \sinh^{-1} \left( Q \frac{x-x_\textrm{min}}{\textrm{stretch}} \right)$$

where `Q` is the same as the $\beta$ softening parameter from [Lupton et al. 2004](https://iopscience.iop.org/article/10.1086/382245) and `stretch` determines the size of the linear region, has been implemented in the [LSST software stack](https://github.com/lsst/afw/blob/master/python/lsst/afw/display/rgb/rgbContinued.py#L282-L321) and also in [astropy.visualization.make_lupton_rgb](http://docs.astropy.org/en/stable/api/astropy.visualization.make_lupton_rgb.html#astropy.visualization.make_lupton_rgb). In this document we make use of the image mapping classes contained in the [lupton_rgb.py](https://github.com/astropy/astropy/blob/master/astropy/visualization/lupton_rgb.py) module in astropy to have a bit more control over how we display our images.

*scarlet* adds the additional functionality of mapping multiband images with *more* than three colors into RGB channels. This is especially useful for deblending, where most ground based telescopes have more than 3 filters. The [img_to_channels](display.ipynb#scarlet.display.img_to_channels) method takes a multiband image `img` as an input and maps all of the bands to RGB colors using the optional `filter_weights` parameter. For example, one band is mapped to all three channels, making a black and white image, while three bands are mapped directly to the RGB channels in reverse order:

In [ ]:
import scarlet.display

print("Mapping from 1 band to RGB:\n", scarlet.display.get_default_filter_weight(1))
print("Mapping from 3 bands to RGB:\n", scarlet.display.get_default_filter_weight(3))

In the typical case, where there are more than 3 bands in an image cube, we see that multiple bands are mapped to the same channel, and similarly a single band may be mapped to multiple channels:

In [ ]:
print("Mapping from 5 bands to RGB:\n", scarlet.display.get_default_filter_weight(5))

This makes it possible to map the *gri* bands in our image to the RGB colors using the Lupton color scaling using

In [ ]:
# Set the arcsinh color scaling object
asinh = AsinhMapping(minimum=images.min(), stretch=1, Q=10)
# Scale the RGB channels for the image
img_rgb = scarlet.display.img_to_rgb(images[:3], norm=asinh)
# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
# Mark each source from the catalog in the image
for k, src in enumerate(catalog):
    plt.text(src["x"], src["y"], str(k), color="r")
plt.show()

This can be a useful visualization to see the color gradients that exist between different bands but we want to make sure that our deblender outputs do not produce residuals in any band. For this we can pass the entire `images` array to [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb), using the same `asinh` normalization, and we get a representation of the entire *grizy* image using the 5-band filter weights given above:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

Of course we might want to use a different mapping of bands to RGB, for example mapping $g \rightarrow B$ and $y \rightarrow R$, so we can define our own filter_weights and pass those to [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb):

In [ ]:
filter_weights = scarlet.display.get_default_filter_weight(5)
filter_weights[0, :] = [0, 0, 0, 0, 1]
filter_weights[2, :] = [1, 0, 0, 0, 0]
filter_weights /= filter_weights.sum(axis=1)[:,None]
print("New filer weights:\n", filter_weights)

img_rgb = scarlet.display.img_to_rgb(images, norm=asinh, filter_weights=filter_weights)
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

The code below shows an example of how to display all of the sources in a model to compare it with the same footprint in the observed images and will be used throughout the remainder of this document. It is beyond the scope of this document to explain in detail how this function works with [matplotlib](https://matplotlib.org) to create the plots, but it is a convenience tool to display all sources and components in a given model:

In [ ]:
# Display the sources
def display_sources(sources, observation, norm=None, subset=None, combine=False, show_sed=True):
    """Display the data and model for all sources in a blend
    
    This convenience function is used to display all (or a subset) of
    the sources and (optionally) their SED's.
    """
    if subset is None:
        # Show all sources in the blend
        subset = range(len(sources))
    for m in subset:
        # Load the model for the source
        src = sources[m]
        if hasattr(src, "components"):
            components = len(src.components)
        else:
            components = 1
        # Convolve the model with the psfs in the observation
        model = observation.render(src.get_model())
        # Extract the bounding box that contains the non-zero
        # pixels in the model
        bbox = scarlet.bbox.trim(np.sum(model, axis=0), min_value=1e-2)
        bb = (slice(None), *bbox.slices)
        # Adjust the stretch based on the maximum flux in the model for the current source
        if model.max() > 10 * bg_rms.max():
            norm = AsinhMapping(minimum=model.min(), stretch=model.max()*.05, Q=10)
        else:
            norm = LinearMapping(minimum=model.min(), maximum=model.max())
        
        # Select the image patch the overlaps with the source and convert it to an RGB image
        img_rgb = scarlet.display.img_to_rgb(images[bb], norm=norm)

        # Build a model for each component in the model
        if hasattr(src, "components"):
            rgb = []
            for component in src.components:
                # Convert the model to an RGB image
                _model = observation.render(component.get_model())
                _rgb = scarlet.display.img_to_rgb(_model[bb], norm=norm)
                rgb.append(_rgb)
        else:
            # There is only a single component
            rgb = [scarlet.display.img_to_rgb(model[bb], norm=norm)]

        # Display the image and model
        figsize = [6,3]
        columns = 2
        # Calculate the number of columns needed and shape of the figure
        if show_sed:
            figsize[0] += 3
            columns += 1
        if not combine:
            figsize[0] += 3*(components-1)
            columns += components-1
        # Build the figure
        fig = plt.figure(figsize=figsize)
        ax = [fig.add_subplot(1,columns,n+1) for n in range(columns)]
        ax[0].imshow(img_rgb)
        ax[0].set_title("Data: Source {0}".format(m))
        for n, _rgb in enumerate(rgb):
            ax[n+1].imshow(_rgb)
            if combine:
                ax[n+1].set_title("Initial Model")
            else:
                ax[n+1].set_title("Component {0}".format(n))
        if show_sed:
            frame = src.frame
            if components > 1:
                for comp in src:
                    ax[-1].plot(comp.sed)
            else:
                ax[-1].plot(src.sed)
            ax[-1].set_xticks(range(frame.C))
            ax[-1].set_xticklabels(frame.channels)
            ax[-1].set_title("SED")
            ax[-1].set_xlabel("Band")
            ax[-1].set_ylabel("Intensity")
        # Mark the current source in the image
        if components > 1:
            y,x = src.components[0].pixel_center
        else:
            y,x = src.pixel_center
        ax[0].plot(x-bb[2].start, y-bb[1].start, 'rx', mew=2)
        plt.tight_layout()
        plt.show()

We can now use this function to display the initial guess for all detected sources in the blend:

In [ ]:
# In order to display the models in the observed seeing
# we have to match the PSFs in the model Scene
# the PSFs in the Observation.
observation = observation.match(frame)
# Show the initial guess for the brightest four sources
display_sources(sources[:4], observation, norm=asinh)

### Hierarchical Sources

In cases where it makes sense to group a collection of components into a single source, for example a galaxy with a bulge and disk, a source class can be derived from [ComponentTree](component.ipynb#scarlet.component.ComponentTree). A component tree is a hierarchical container with all of the same methods as [Component](component.ipynb#scarlet.component.Component), only methods like [get_flux](component.ipynb#scarlet.component.ComponentTree.get_fux) and [get_model](component.ipynb#scarlet.component.ComponentTree.get_model) combine the results from all of the children in the tree.

A sample [MultiComponentSource](source.ipynb#scarlet.source.MultiComponentSource) class is included in *scarlet* and gives an example of one way to initialize a bulge-disk galaxy, however this class has not been adequately tested and should only be used as a starting point to derive your own multi-component source class. If you come up with a class that works better please submit a pull request to add it to *scarlet*, we would be happy to include it!

The example below models all of the sources with two components and displays the initial:

In [ ]:
sources = [
    scarlet.MultiComponentSource(frame, (src['y'],src['x']), observation, bg_rms)
    for src in catalog
]
# Show the initial guess for the brightest four sources
display_sources(sources[:4], observation, norm=asinh)

As you can see from the plots, there are still improvements needed to the [MultiComponentSource](source.ipynb#scarlet.source.MultiComponentSource) class but this should illustrate how a multi-component source can be created and displayed.

## Updates and Constraints

Components conatain gradients in the `sed_grad` and `morph_grad` attributes that are updated by [Blend](blend.ipynb#scarlet.blend.Blend) in each iteration. Once the gradient updates have been applied to all of the components blend will call [Component.update](component.ipynb#scarlet.component.Component.update) on each component to apply any constraints.

All components should have at least one minimal constraints that is necessary to break degeneracies in the model. For example, given the basic model of the scene 

$$\mathsf{M}= \sum_{k=1}^K \mathsf{A}_k^T \times \mathsf{S}_k = \mathsf{A}\mathsf{S}, $$

there are an infinite number of solutions that can be obtained by multiplying a column in $\mathsf{A}$ by some number $\alpha$ and dividing its corresponding row in $\mathsf{S}$ by the same number $\alpha$. To break this degeneracy we require some normalization proceedure to give the gradients a more defined direction. As mentioned in the [Introduction](#Basic-Concepts-and-Structure), constraints are applied as proximal operators which can simplistically be thought of as projections of a model parameter to the space where the constraint is met. For example, we can define the SED to normalize to one by dividing the SED by `component.sed.sum()` and multiplying `component.morph` by the same value. We'll discuss this further in [Normalization](#Normalization).

### Constructing a new Constraint

The [proxmin](https://github.com/pmelchior/proxmin) package contains many of the basic proximal operators required, with additional proximal operators found in the [operator](operator.ipynb) module along with more advanced update functions in the [update](update.ipynb) module. Perhaps the best way to illustrate how to use constraints is to re-create a basic update method contained in *scarlet*.

Since flux emitted by sources should typically be non-negative (for now we ignore absorption by dust which would be governed by more complicated constraints) we can project all negative values in the `morph` and `sed` to zero. By convention we define each update function to accept a [Component](component.ipynb#scarlet.component.Component) as the first argument, update or create attributes of that method in place, and return the modified [Component](component.ipynb#scarlet.component.Component). So the function to enforce non-negativity on the sed is just

In [ ]:
# Load the non-negativity proximal operator from proxmin
from proxmin.operators import prox_plus

def positive_sed(component):
    """Make the SED non-negative
    
    An example of an update function to apply a proximal operator.
    By convention update functions always take a `component` as an
    input, update the component in place, and return the updated
    component.
    """
    prox_plus(component.sed, component.step_sed)
    return component

Now we can create a source that only uses our new constraint by inheriting from `PointSource` and overloading the `update` method:

In [ ]:
import scarlet.update as update
class MinimallyConstrainedSource(scarlet.PointSource):
    def update(self):
        """A Source with only normalization and non-negativity
        """
        # Make the sed non-negative
        positive_sed(self)
        # Make the morphology non-negative
        update.positive_morph(self)
        # Normalize the SED to sum to unity
        update.normalized(self, 'sed')

### Normalization

As mentioned [above](#Updates-and-Constraints), in order to break the degeneracy between the sed and morphology we need to normalize one of the two parameters. While the user may use any normalization that suits his/her needs, the [normalized](update.ipynb#normalized) method implements several useful normalization schemes

1. `sed`
2. `morph`
3. `morph_max`

Passing `type="sed"` to [normalized](update.ipynb#normalized) normalizes the `sed` so that its elements sum to unity, making `sed` a normalized SED while `morph` contains all of of the shape and intensity information. This has the advantage that the SED, which converges much faster when it is normalized, is stable for the majority of the iterations. The disadvantage is that overlapping sources that are roughly the same color but have very different intensities (which is common in galaxy clusters, for example) can be difficult to separate with this normalization.

Using `type="morph"` normalizes the `morph` matrix so that it sums to unity for each source, so that `sed` now contains both color and intensity information. The advantage to this method is that two sources with very similar colors are easier to distinguish if they have very different intensities. The other big advantage of this normalization is that true point sources can be fit much more easily and efficiently by fixing the morphology and normalizing `morph`, meaning the intensity of the central pixel equals 1 for all iterations and only the colors and intensities in `sed` are modified.

The main disadvantage of this method is that using `type="morph"` for an [ExtendedSource](#scarlet.source.ExtendedSource) causes fluctuations in the values of the outer pixels during fitting to affect the overall global normalization of `morph` for each source, meaning a peak pixel that is relatively well constrained will not be fixed if a large number of pixels on the outer edges of the source change value (which can happen while fitting regions with low signal to noise, especially if they overlap with another source).

To limit this affect we provide the `type="morph_max"` normalization, which normalizes `morph` so that the peak is always set to a value of 1. This should make the values near the peak more robust and usually results in faster convergence and lower residuals compared to the other two normalization schemes. For this reason it is the default normalization in *scarlet*.

For more on modeling points sources see the [Point Source Tutorial](tutorials/point_source.ipynb).

### Symmetry

Demanding that astrophysical sources are symmetric reduces the number of effective degrees of freedom of the model, and most galaxies are *largely* symmetric. The idea of using symmetry as a constraint has been used successfully in the SDSS deblender and also in our tests on substantially deeper HSC images. If this constraint feels overly restrictive, recall that virtually all galaxy model fitting impose symmety using more restrictive Sersic profiles, which also make assumptions about the radial profile.

To make a source symmetric requires a position to make the model symmetric about. Source models are highy sensitive to this fractional pixel location so it is necessary to include an update function that estimates the position of a symmetric source in the blend. See [Center Positions](#Center-Positions) for more on setting the center of a source.

### Monotonicity

Another useful constraint from the SDSS-HSC deblender is the approximation that most astrophysical objects are monotonically decreasing from the peak.
In detail this assumption is violated e.g. in spiral galxies, especially tightly wound ones.
But if we think of spirals as a single source made up of multiple components, each monotonically decreasing from it's peak with a single SED, we can build a model that is well representative of even morphologically complex galaxies.
This point of view has the added benefit that regions that are not monotonically decreasing in a galaxy are likely different stellar populations with (potentially) different SED's and should be treated as separate components anyway.

In *scarlet* monotonicity is implemented as a projection that is *not* a true proximal operator (while it is possible to write monotonicity as a true proximal operator, the implementation of the algorithm is far too expensive for any practical deblending purpose). Instead the morphology is projected into a space that has *a* monotonic solution, just not the ideal one. There are two possible monotonic solutions to use. If `use_nearest=True` then only a single reference pixel is used: the nearest one in the direction to the peak.
Otherwise a weighted average of all pixels closer to the peak than the current pixel is used to allow for a smoother monotonic solution.

| ![](images/nearest_ref.png) | ![](images/weighted_ref.png) |
|:---------------------------:|:----------------------------:|
| Nearest Neighbor            | Weighted Reference           |


The following example illustrates how to implement monotonicity for a source. It is important to update the center of a monotonic source often, otherwise an initially improperly centered source will never be able to accurately model the underlying flux. 

In [ ]:
class MonotonicSource(scarlet.Component):
    """A source with flux that montonically decreases from its center
    """
    def __init__(self, use_nearest, *args, **kwargs):
        self.use_nearest = use_nearest
        super.__init__(*args, **kwargs)

    def update(self):
        # Find the pixel in the morphology with maximum fluxand use it as the center
        update.fit_pixel_center(self)
        # Make the model monotonic
        update.monotonic(self, self.pixel_center, use_nearest=self.use_nearest)
        # Make the sed and morphology non-negative
        update.positive(self)
        # Normalize the morphology to have a peak value of unity
        update.normalized(self, 'morph_max')
        

### Center Positions

In order to model individual sources in an image it is usually necessary to give each source a center position (for jets and irregular galaxies the concept of "center" is a bit less defined). Traditional photometric measurements typicaly define the center by the weighted center of flux calculation (commonly abusing terminology and refering to the position as a *centroid*, not to be confused with the *geometric centroid*). This location can be misleading because in a blend the center of flux for blended objects will naturally be shifted toward any neighboring sources, so naively using the center of flux for a source in the observations will be insufficient.

In many cases the location of the peak pixel is sufficient (e.g. [monotonicity](#Monotonicity)) without worrying about the subpixel location. In these cases the [fit_pixel_center](update.ipynb#scarlet.update.fit_pixel_center) method can be used to find the pixel in `component.morph` with the maximum flux and use that to set `component.pixel_center`. It is possible to pass a `window` to [fit_pixel_center](update.ipynb#scarlet.update.fit_pixel_center), where `(0, 0)` is the center of the morphology, and only use the flux inside that window to determine the pixel center. By default `window = (slice(cy-2, cy+3), slice(cx-2, cx+3))`, meaning only pixels in a $7\times 7$ window centered on the estimated center position are used to search for the peak pixel. This prevents bright sources on the outskirts of fainter ones from causing the pixel center to jump from the fainter source to a pixel in the brighter source.

When a more precise center is needed the center of flux of the *morphology* can be used to estimate the center of the source. This measurement is different than the center of flux from the observation images, since the model should not have contamination from neighboring sources and ideally use a target PSF with circular symmetry.

## Blended scenes

The [Blend](blend.ipynb#scarlet.blend.Blend) class combines the sources and observations, and has all methods to fit the model to the data data. Internally it organizes the components as a tree that has the same access pattern to its components as [ComponentTree](component.ipynb#scarlet.component.ComponentTree).

### Initialization

Initializing a new blended scene requires to specify a list of `sources` ([Component](component.ipynb#scarlet.component.Component) or [ComponentTree](component.ipynb#scarlet.component.ComponentTree)  objects) and a set of `observations` (either a list of or a single [Observation](observation.ipynb#Observation)). `Blend` will inherit its `Frame` from `sources`.

In [ ]:
sources = [scarlet.source.ExtendedSource(frame, (src['y'],src['x']), observation, bg_rms) for src in catalog]
blend = scarlet.Blend(sources, observation)

### Fitting a Model

The [Blend](blend.ipynb#scarlet.blend.Blend) class uses the [autograd](https://github.com/HIPS/autograd) package to calculate the gradient of the model, then applies any constraints or updates to the model parameters.

The [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) method will fit the current model and has three parameters: the maximum number of steps (or iterations) `max_iter` used to fit the data, the relative error for convergence `e_rel`, and whether or not to approximate the Lipschitz constant `approximate_L`, which can make the code run faster per iteration at the risk of using suboptimal step sizes.
It stops if one of the following conditions is met:

1. The total number of iterations is equal to `max_iter`

1. All of the parameters in the model converge, with changes smaller than `e_rel` in the last iteration


### Restarting a Fit

There may be instances where it is desirable to restart a fit.
For example, after a certain number of iterations inspect the result even prior to convergence, or you may have a custom constraint that you want to apply every Nth iteration.
In that case you can call `Blend.fit(N1)` and continue with another call to `Blend.fit(N2)`:

In [ ]:
blend.fit(10)
print(blend.it) 
blend.fit(10)
print(blend.it)
blend.fit(10)
print(blend.it)

where we see that the blend converged in the last `fit` before reaching 30 iterations.

### Extracting Models from a Blend

Just like we did with the sources [above](#Displaying-a-model), we can extract the model from a blend using

In [ ]:
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
model_rgb = scarlet.display.img_to_rgb(model, norm=asinh)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
axes[0].imshow(img_rgb)
axes[1].imshow(model_rgb)
plt.show()

Notice that the result does *not* look very much like the model. This is because when `get_model` is called from either a [Component](component.ipynb#scarlet.component.Component) or a [Blend](blend.ipynb#scarlet.blend.Blend) it returns it in the model frame (both in terms of resolution and target PSF). To convert the model to the observation frame we need to use [Observation.render](observation.ipynb#scarlet.observation.Observation.render):

In [ ]:
model = blend.get_model()
model_ = observation.render(model)
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
model_rgb = scarlet.display.img_to_rgb(model_, norm=asinh)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
axes[0].imshow(img_rgb)
axes[1].imshow(model_rgb)
plt.show()

It is also possible to load the model for each source individually:

In [ ]:
for src in blend.sources:
    model = src.get_model()
    model_ = observation.render(model)
    if model.max() > 10 * bg_rms.max():
        norm = AsinhMapping(minimum=0, stretch=model_.max()*.05, Q=10)
    else:
        norm = LinearMapping(minimum=0, maximum=model_.max())
    model_rgb = scarlet.display.img_to_rgb(model_, norm=norm)
    plt.imshow(model_rgb)
    plt.show()

We can also display all of the components for a given blend, which is useful when one or more sources have multiple components. In this case, because each source was initialized with a single component, there is no difference:

In [ ]:
for component in blend.components:
    model = component.get_model()
    model_ = observation.render(model)
    if model.max() > 10 * bg_rms.max():
        norm = AsinhMapping(minimum=0, stretch=model_.max()*.05, Q=10)
    else:
        norm = LinearMapping(minimum=0, maximum=model_.max())
    model_rgb = scarlet.display.img_to_rgb(model_, norm=norm)
    plt.imshow(model_rgb)
    plt.show()

## Checking for Flags and Convergence

### BlendFlag

The [BlendFlag](component.ipynb#scarlet.component.BlendFlag) class is used to keep track of any potential problems that may have arisen during deblending. The two most useful kesy are `BlendFlag.SED_NOT_CONVERGED` and `BlendFlag.MORPH_NOT_CONVERGED`, which indicate that the SED and morphology have not converged respectively. This can be useful when testing out a new source class or set of observations to understand why blends are not converging as expected.

While not implemented yet, [BlendFlag](component.ipynb#scarlet.component.BlendFlag) can also indicate whether a source had pixels on the edge of the frame in the model scene, which means any shapes or fluxes measured might be incorrect, and whether or not the source had zero flux.

### Convergence

Convergence of the entire blend can also be tested using the [Blend.converged](blend.ipynb#scarlet.blend.Blend.converged) property, which is only true if the SED and morphology have converged for every source.

Putting these concepts together we can create a `History` class that allows us to track the convergence of our sources in each iteration:

In [ ]:
class History(scarlet.ExtendedSource):
    def __init__(self, *args, **kwargs):
        # The location of the center in each iteration
        self.center_history = []
        # The convergence in each iteration
        self.convergence_hist = []
        # The number of each iteration
        self.iterations = []
        super().__init__(*args, **kwargs)

    def update(self):
        """Update the history and the parameters
        """
        if self._parent is None:
            it = 0
        else:
            it = self._parent.it
        self.update_history(it)
        return super().update()
    
    def update_history(self, it):
        """Update the center and convergence history
        """
        self.center_history.append(self.pixel_center)
        self.iterations.append(it)
        _sed = self.flags & scarlet.BlendFlag.SED_NOT_CONVERGED
        _morph = self.flags & scarlet.BlendFlag.MORPH_NOT_CONVERGED
        # Add a color that can be used by the plot to indicate which paremeters
        # have converged
        if _sed and _morph:
            self.convergence_hist.append("black")
        elif _sed:
            self.convergence_hist.append("cyan")
        elif _morph:
            self.convergence_hist.append("red")
        else:
            self.convergence_hist.append("green")

sources = [History(frame, (src['y'],src['x']), observation, bg_rms) for src in catalog]

With our new source list we can deblend again and then track the convergence of each source by plotting its pixel center in x and y. Black dots indicate that neither the SED nor the morphology converged for a source, red dots indicate that only the SED converged for a source, cyan dots indicate that only the morphology converged, and green dots indicate that both the SED and morphology converged for a source in a given iteration:

In [ ]:
# Create and fit the blend
blend = scarlet.Blend(sources, observation)
blend.fit(100, 1e-2, False)
print("Total iterations:", blend.it)

# Show the history of pixel positions and convergence
for k,src in enumerate(blend.sources):
    history = np.array(src.center_history)
    hy = history[:, 0]
    hx = history[:, 1]
    fig = plt.figure(figsize=(12,3 ))
    ax = [fig.add_subplot(1, 2, n+1) for n in range(2)]
    x = src.iterations
    ax[0].scatter(x, hy, c=src.convergence_hist, s=5)
    ax[0].set_title("py")
    ax[1].scatter(x, hx, c=src.convergence_hist, s=5)
    ax[1].set_title("px")
    ax[0].set_ylim([hy[-1]-1, hy[-1]+1])
    ax[1].set_ylim([hx[-1]-1, hx[-1]+1])

    plt.show()

Notice that for most sources it was the morphology that converged first and it was the faintest source that converged last. This makes sense: the morphologies are normalized and most of their flux is located in the center, so tose values tend to converge faster, and sources with low signal to noise struggle to find an amplitude consistent with the data when their neighbors with much larger flux are constantly adjusting the outer edges of their models.

This concludes the overview. The user is referred to the [API Documentation](api_docs.rst) for more details about the objects used in *scarlet*.